## Лабораторная работа №2

#### Задание 1
Провести классификацию найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

#### Ход работы:

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Загрузка данных
data = pd.read_csv('universal_top_spotify_songs.csv')

# Вывод общей информации о данных
print("Общая информация о данных:")
print(f"Количество строк: {data.shape[0]}")
print(f"Количество столбцов: {data.shape[1]}")
print("\nТипы данных по столбцам:")
print(data.dtypes)

Общая информация о данных:
Количество строк: 10000
Количество столбцов: 20

Типы данных по столбцам:
spotify_id           object
daily_rank            int64
daily_movement        int64
weekly_movement       int64
country              object
popularity            int64
is_explicit            bool
duration_ms           int64
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
dtype: object


In [ ]:
# Отделение признаков и целевой переменной
X = data.drop(columns=['popularity'])  # Предполагается, что 'Medical Condition' - целевая переменная
y = data['popularity']

# Кодирование категориальных переменных
categorical_cols = ['spotify_id', 'country']
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))

# Обработка отсутствующих значений и масштабирование
X.fillna(0, inplace=True)  # Заполнение отсутствующих значений нулями в качестве примера, выберите соответствующий метод
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Указание гиперпараметров и их значений для поиска
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10], 'kernel': ['linear', 'rbf', 'sigmoid', 'poly'], 'gamma': [0.1, 1, 10], 'degree': list(range(2, 9))}

# Создание объекта GridSearchCV
grid_search = GridSearchCV(SVC(), param_grid, cv=5)

# Подгонка сетки по данным
grid_search.fit(X_train, y_train)

# Получение лучших параметров и лучшего оценщика
best_params = grid_search.best_params_
best_svm = grid_search.best_estimator_

print(f'Лучшие параметры: {best_params}')

# Создание прогнозов с использованием лучшего оценщика
y_pred = best_svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy}")

# Получение отчета о классификации с предупреждениями
report = classification_report(y_test, y_pred, zero_division=1)
print(f'Отчет о классификации:\n{report}')

# Определение отсутствующих классов в прогнозах
classes = np.unique(y_test)
predicted_classes = np.unique(y_pred)


Лучшие гиперпараметры: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Точность модели: 0.8171666666666667
Отчет о классификации:
              precision    recall  f1-score   support

           0       0.83      0.95      0.89      4687
           1       0.67      0.33      0.44      1313

    accuracy                           0.82      6000
   macro avg       0.75      0.64      0.66      6000
weighted avg       0.80      0.82      0.79      6000

